In [18]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras import Model
from keras.models import Sequential, load_model
from keras.layers import Dense, SimpleRNN, Dropout, GRU, Layer, Input, Convolution2D, Convolution1D, AveragePooling2D, MaxPool2D, Flatten, ConvLSTM1D
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [19]:
df = pd.read_csv("train.csv")
df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df.shape

(42000, 785)

In [21]:
y = df["label"] 
y = tf.keras.utils.to_categorical(y, num_classes=10, dtype='float32')
X = df.drop(columns={"label"})
X.shape

(42000, 784)

In [22]:
X = X / 255 
X

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
X = np.array(np.split(X.values, 28, axis=1))
X

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [24]:
X = X.transpose((1,0,2))

In [25]:
X.shape

(42000, 28, 28)

In [26]:
X = np.expand_dims(X, axis = 3)
X.shape

(42000, 28, 28, 1)

In [27]:
image_gen = ImageDataGenerator(rotation_range=5, zoom_range=0.05, width_shift_range=0.05, height_shift_range=0.05,
                               validation_split=0.2, horizontal_flip=False, vertical_flip= False, 
                               data_format="channels_last") # 80/20 train/val split

train_generator = image_gen.flow(X, 
                                 y,
                                batch_size=20,
                                shuffle=True,
                                subset='training',
                                seed=42)
valid_generator = image_gen.flow(X,
                                 y,
                                batch_size=20,
                                shuffle=True,
                                subset='validation')

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, 
                                                    train_size=0.8, random_state=262)
X_train

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [29]:
X_train.shape

(33600, 28, 28, 1)

In [36]:
inputs = Input(shape=[28,28,1])

x1 = Convolution2D(filters=32, kernel_size=(5,10), activation="LeakyReLU", padding="same")(inputs)
x2 = MaxPool2D(pool_size=(2, 2))(x1)
x3 = Convolution2D(filters=32, kernel_size=(10,5), activation="LeakyReLU", padding="same")(x2)
x4 = MaxPool2D(pool_size=(2, 2))(x3)
x5 = Convolution2D(filters=32, kernel_size=(6,2), activation="LeakyReLU", padding="same")(x4)
x6 = MaxPool2D(pool_size=(2, 2))(x5)
x7 = Convolution2D(filters=32, kernel_size=(2,6), activation="LeakyReLU", padding="same")(x6)
x8 = Flatten()(x7)

x9 = Dense(20, "LeakyReLU")(x8)

outputs = Dense(10, activation="softmax", input_shape=[x9.shape])(x9) 

CNN = Model(inputs=inputs, outputs=outputs, name="CNN")
CNN.compile(loss=["kl_divergence"], optimizer="nadam", metrics=["accuracy"])
memory = CNN.fit(train_generator, batch_size = 20, validation_data=(valid_generator), 
                 epochs=15, use_multiprocessing=True)

Epoch 1/15
1680/1680 [==============================] - 30s 17ms/step - loss: 0.2117 - accuracy: 0.9327 - val_loss: 0.0736 - val_accuracy: 0.9761
Epoch 2/15
1680/1680 [==============================] - 29s 17ms/step - loss: 0.0785 - accuracy: 0.9754 - val_loss: 0.0721 - val_accuracy: 0.9795
Epoch 3/15
1680/1680 [==============================] - 29s 17ms/step - loss: 0.0605 - accuracy: 0.9812 - val_loss: 0.0665 - val_accuracy: 0.9801
Epoch 4/15
1680/1680 [==============================] - 29s 17ms/step - loss: 0.0574 - accuracy: 0.9819 - val_loss: 0.0540 - val_accuracy: 0.9832
Epoch 5/15
1680/1680 [==============================] - 30s 18ms/step - loss: 0.0499 - accuracy: 0.9846 - val_loss: 0.0518 - val_accuracy: 0.9849
Epoch 6/15
1680/1680 [==============================] - 29s 17ms/step - loss: 0.0447 - accuracy: 0.9861 - val_loss: 0.0770 - val_accuracy: 0.9780
Epoch 7/15
1680/1680 [==============================] - 29s 17ms/step - loss: 0.0451 - accuracy: 0.9868 - val_loss: 0.0491 -

In [37]:
CNN.summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_44 (Conv2D)          (None, 28, 28, 32)        1632      
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_45 (Conv2D)          (None, 14, 14, 32)        51232     
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 7, 7, 32)         0         
 g2D)                                                            
                                                                 
 conv2d_46 (Conv2D)          (None, 7, 7, 32)          12320   

In [38]:
df1 = pd.read_csv("test.csv")
df1

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df1 = df1 / 255 
df1 = np.array(np.split(df1.values, 28, axis=1))
df1 = df1.transpose((1,0,2))
df1.shape

(28000, 28, 28)

In [40]:
df2 = pd.read_csv("sample_submission.csv", index_col="ImageId")
df2

,Label
ImageId,
1,0
2,0
3,0
4,0
5,0
...,...
27996,0
27997,0
27998,0


In [41]:
y_pred = CNN.predict(df1)
y_pred.shape

(28000, 10)

In [42]:
labels = []
for i in range(28000):
    y_pred_label = np.argmax(y_pred[i,:])
    labels.append(y_pred_label)
labels

[2,
 0,
 9,
 0,
 3,
 7,
 0,
 3,
 0,
 3,
 5,
 7,
 4,
 0,
 4,
 3,
 3,
 1,
 9,
 0,
 9,
 1,
 1,
 5,
 7,
 4,
 2,
 7,
 4,
 7,
 7,
 5,
 4,
 2,
 6,
 2,
 5,
 5,
 1,
 6,
 7,
 7,
 4,
 9,
 8,
 7,
 8,
 2,
 6,
 7,
 6,
 8,
 8,
 3,
 8,
 2,
 1,
 2,
 2,
 0,
 4,
 1,
 7,
 0,
 0,
 0,
 1,
 9,
 0,
 1,
 6,
 5,
 8,
 8,
 2,
 8,
 9,
 9,
 2,
 3,
 5,
 4,
 1,
 0,
 9,
 2,
 4,
 3,
 6,
 7,
 2,
 0,
 6,
 6,
 1,
 4,
 3,
 9,
 7,
 4,
 0,
 9,
 2,
 0,
 7,
 3,
 0,
 5,
 0,
 8,
 0,
 0,
 4,
 7,
 1,
 7,
 1,
 1,
 3,
 3,
 3,
 7,
 2,
 8,
 6,
 3,
 8,
 7,
 8,
 4,
 3,
 5,
 6,
 0,
 0,
 0,
 3,
 1,
 3,
 6,
 4,
 3,
 4,
 5,
 5,
 8,
 7,
 7,
 2,
 8,
 4,
 3,
 5,
 6,
 5,
 3,
 7,
 5,
 7,
 8,
 3,
 0,
 4,
 5,
 1,
 3,
 7,
 6,
 3,
 0,
 2,
 7,
 8,
 6,
 1,
 3,
 7,
 4,
 1,
 2,
 4,
 8,
 5,
 2,
 4,
 9,
 2,
 1,
 6,
 0,
 6,
 1,
 4,
 9,
 6,
 0,
 9,
 7,
 6,
 9,
 1,
 9,
 0,
 9,
 9,
 0,
 8,
 4,
 6,
 2,
 0,
 9,
 3,
 6,
 3,
 2,
 1,
 6,
 3,
 4,
 2,
 3,
 1,
 2,
 2,
 0,
 4,
 6,
 1,
 0,
 0,
 4,
 9,
 1,
 7,
 3,
 2,
 3,
 8,
 6,
 8,
 6,
 2,
 8,
 5,
 5,
 4,
 8,
 3,
 5,


In [43]:
len(labels)

28000

In [44]:
df2["Label"] = labels
df2

,Label
ImageId,
1,2
2,0
3,9
4,0
5,3
...,...
27996,9
27997,7
27998,3


In [45]:
df2.to_csv("predicted_model4.csv")